In [2]:
import matplotlib.pyplot as plt
import cv2
import scipy.signal as signal
import numpy as np

In [3]:
class Kinect(object):
    def __init__(self, Kinect_data):
        self.raw_data = Kinect_data

    def compute_num_timestamp(self):
        count_num = int(len(self.raw_data['time'])/25)

        return count_num

    def get_xyz_tolist(self):
        key_points_data = []
        for n in range(25):
            key_points_data.append([])
            for m in range(3):
                key_points_data[n].append([])

        for k in range(25):
            for m in range(self.compute_num_timestamp()):
                key_points_data[k][0].append(self.raw_data.iloc[m*25+k, 2])
                key_points_data[k][1].append(self.raw_data.iloc[m*25+k, 3])
                key_points_data[k][2].append(self.raw_data.iloc[m*25+k, 4])

        return key_points_data

    def get_timestamp(self):
        timestamp = []
        for t in range(self.compute_num_timestamp()):
            timestamp.append(self.raw_data.iloc[t*25, 6])

        return timestamp

    def get_maxtime(self):
        return max(self.get_timestamp())

In [4]:
class Kinect_split(Kinect):
    def __init__(self, Kinect_data):
        Kinect.__init__(self, Kinect_data)

    # 使用median5 和 Gaussian15，5对keypoint某一维数据过滤噪声
    def noise_reduction(self, oneD_Kdata):
        oneD_Kdata_mean = cv2.blur(np.array(oneD_Kdata), (1,5))
        oneD_Kdata_mean_gauss = cv2.GaussianBlur(oneD_Kdata_mean, (1,15), 5)
        oneD_Kdata_mean_gauss = np.squeeze(oneD_Kdata_mean_gauss)
        oneD_Kdata_mean_gauss_mean = cv2.blur(oneD_Kdata_mean_gauss, (1,5))

        return oneD_Kdata_mean_gauss_mean

    def sb_noise_reduction(self, oneD_Kdata):
        oneD_Kdata_mean = cv2.blur(np.array(oneD_Kdata), (1,5))
        oneD_Kdata_mean_gauss = cv2.GaussianBlur(oneD_Kdata_mean, (1,25), 10)
        oneD_Kdata_mean_gauss = np.squeeze(oneD_Kdata_mean_gauss)
        oneD_Kdata_mean_gauss_mean = cv2.blur(oneD_Kdata_mean_gauss, (1,5))
        return oneD_Kdata_mean_gauss_mean

    #  对spinebase数据滤波并返回一个数据列表
    def sb_z(self):
        sb_z_filt = self.sb_noise_reduction(self.get_xyz_tolist()[0][2])
        sb_z_filt = sb_z_filt.reshape(-1)
        sb_z_filt = sb_z_filt.tolist()

        return sb_z_filt

    #  对left&right elbow数据滤波并差分取绝对值，返回一个滤波后的列表
    def elbow_diff(self):
        elbow_left_filt = self.noise_reduction(self.get_xyz_tolist()[5][0]).reshape(-1).tolist()
        elbow_right_filt = self.noise_reduction(self.get_xyz_tolist()[9][0]).reshape(-1).tolist()
        elbow_diff = [abs(elbow_left_filt[i]-elbow_right_filt[i]) for i in range(len(elbow_left_filt))]
        elbow_diff = self.noise_reduction(elbow_diff).reshape(-1).tolist()

        return elbow_diff

    #  对left&right foot数据滤波并差分，返回一个滤波后的列表
    def foot_diff(self):
        foot_left_filt = self.noise_reduction(self.get_xyz_tolist()[15][2]).reshape(-1).tolist()
        foot_right_filt = self.noise_reduction(self.get_xyz_tolist()[19][2]).reshape(-1).tolist()
        foot_diff = [(foot_left_filt[i]-foot_right_filt[i]) for i in range(len(foot_left_filt))]
        foot_diff = self.noise_reduction(foot_diff).reshape(-1).tolist()

        return foot_diff

    #  获得行走阶段的起止点数据
    def get_walkingpoint(self):
        sb_z_filt = self.sb_z()

        sb_z_filt_diff = []
        for i in range(len(sb_z_filt)-1):
            sb_z_filt_diff.append(round((sb_z_filt[i+1]-sb_z_filt[i]), 5))

        walking_start_index = 0
        for i in sb_z_filt_diff:
            if i < -0.0050:
                walking_start_index = sb_z_filt_diff.index(i)
                break

        walking_end_index = 0
        for i in sb_z_filt_diff[-int(0.25*len(sb_z_filt_diff)):]:
            if i < 0.0050:
                walking_end_index = sb_z_filt_diff.index(i)
                break

        return walking_start_index, walking_end_index

    #  获得转身阶段的起止点数据
    def get_turningpoint(self):
        elbow_diff = self.elbow_diff()

        turning_start_index = 0
        for i in range(int(0.25*len(elbow_diff)),len(elbow_diff)-1):
            if (round((elbow_diff[i+1]-elbow_diff[i]), 5) < -0.0035) and (0.350 < elbow_diff[i] < 0.500):
                turning_start_index = i
                break

        turning_end_index = 0
        if turning_start_index+20 < len(elbow_diff): 
            for i in range(turning_start_index+20,len(elbow_diff)-1):
                if (elbow_diff[i+1]-elbow_diff[i]) < 0.0040 and (0.350 < elbow_diff[i] < 0.500):
                    turning_end_index = i
                    break

        return turning_start_index, turning_end_index